## 1. 라이브러리 가져오기
## Import Library

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import os
from pathlib import Path

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

In [2]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [3]:
# 날짜 처리
data = pd.read_csv('data/201901-202003.csv')
#data = pd.read_csv(os.path.join(Path(self).resolve().parent, '201901-202003.csv')) 
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [4]:
data.head(5)

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [5]:
data.tail(5)

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
24697787,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,2020,3
24697788,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,2020,3
24697789,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,2020,3
24697790,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,2020,3
24697791,충북,충주시,휴양콘도 운영업,충북,충주시,60s,1,5,3,194000,3,2020,3


In [4]:
# 데이터 정제 + 컬럼 추가

'''
# 추가해볼 만한 컬럼
# 거주 지역과 결제 지역이 같으면 1, 다르면 0
# CNT를 min max로 나눠서 10개 카테고리로 나눠서 해보기
# 업종에 나이가 영향을 줄 수 있지 않을까
# feature 상관분석, PCA 해보기
'''

df = data.copy()

# 1. 계절 / 봄(1), 여름(2), 가을(3), 겨울(4)
df.loc[(df['month']>=3)&(df['month']<=5) ,'season'] = 1
df.loc[(df['month']>=6)&(df['month']<=8) ,'season'] = 2
df.loc[(df['month']>=9)&(df['month']<=11) ,'season'] = 3
df.loc[(df['month']>11),'season'] = 4
df.loc[(df['month']<=2),'season'] = 4

df

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month,season
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1,4.0
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1,4.0
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1,4.0
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1,4.0
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,2020,3,1.0
24697788,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,2020,3,1.0
24697789,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,2020,3,1.0
24697790,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,2020,3,1.0


In [5]:
# 2. 거주(1) or 여행(0)
df.loc[(df['CARD_SIDO_NM'] == df['HOM_SIDO_NM']), 'visitor'] = 1
df.loc[(df['CARD_SIDO_NM'] != df['HOM_SIDO_NM']), 'visitor'] = 0

df

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month,season,visitor
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1,4.0,1.0
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1,4.0,1.0
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1,4.0,1.0
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1,4.0,1.0
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,2020,3,1.0,1.0
24697788,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,2020,3,1.0,1.0
24697789,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,2020,3,1.0,1.0
24697790,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,2020,3,1.0,1.0


In [6]:
# 3. 결제 건수 20개 범위로 나누기 
df['CNT_Category'] = pd.cut(df['CNT'], 20, labels=False)

df

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month,season,visitor,CNT_Category
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1,4.0,1.0,0
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1,4.0,1.0,0
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1,4.0,1.0,0
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1,4.0,1.0,0
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1,4.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,2020,3,1.0,1.0,0
24697788,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,2020,3,1.0,1.0,0
24697789,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,2020,3,1.0,1.0,0
24697790,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,2020,3,1.0,1.0,0


In [9]:
# CNT 처리하기 위해 정리한 것
# cut은 범위를 동일하게 자른 것, qcut은 범위 내에 들어오는 갯수가 동일하게 자른 것

# print(max(df['CNT']))
# print(min(df['CNT']))

# df['CNT']

# pd.cut(df['CNT'], 20, labels=False)

# cnt_category = pd.qcut(df['CNT'], 20, labels=False)
# print(cnt_category)
# print(cnt_category[0])
# print(cnt_category[1])
# print(cnt_category[1057391])

# cnt_category = pd.cut(df['CNT'], 20, labels=False)
# print(max(cnt_category)) # 19
# print(min(cnt_category)) # 0
# print(len(df['CNT']))
# print(cnt_category)

In [7]:
# 4. 코로나
covid = ['관광 민예품 및 선물용품 소매업','그외 기타 분류안된 오락관련 서비스업','그외 기타 스포츠시설 운영업','기타 대형 종합 소매업','기타 수상오락 서비스업','기타 외국식 음식점업','기타 주점업','내항 여객 운송업','마사지업','면세점','버스 운송업','비알콜 음료점업','서양식 음식점업','스포츠 및 레크레이션 용품 임대업','여관업','여행사업','욕탕업','일반유흥 주점업','일식 음식점업','자동차 임대업','전시 및 행사 대행업','정기 항공 운송업','중식 음식점업','차량용 가스 충전업','차량용 주유소 운영업','체인화 편의점','택시 운송업','피자 햄버거 샌드위치 및 유사 음식점업','한식 음식점업','호텔업','화장품 및 방향제 소매업','휴양콘도 운영업']
df['covid'] = False
for virus in covid:
    df.loc[(df['STD_CLSS_NM'] == virus),'covid'] = True

df

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month,season,visitor,CNT_Category,covid
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1,4.0,1.0,0,False
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1,4.0,1.0,0,False
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1,4.0,1.0,0,False
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1,4.0,1.0,0,False
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1,4.0,1.0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,2020,3,1.0,1.0,0,True
24697788,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,2020,3,1.0,1.0,0,True
24697789,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,2020,3,1.0,1.0,0,True
24697790,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,2020,3,1.0,1.0,0,True


In [8]:
# 0. 컬럼 드랍
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

# 0. 독립 변수 선정
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month', 'visitor', 'CNT_Category', 'covid']
df = df.groupby(columns).sum().reset_index(drop=False)

In [9]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

## 3. 탐색적 자료분석
## Exploratory Data Analysis

In [16]:
# 시도 컬럼 수

print(len(data.CARD_SIDO_NM))

sido = pd.Series.unique(data.CARD_SIDO_NM)
print(len(sido))
print(sido)

24697792
17
['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']


In [18]:
# 시군구 컬럼 수

print(len(data.CARD_CCG_NM))

ccg = pd.Series.unique(data.CARD_CCG_NM)
print(len(ccg))
print(ccg)

24697792
227
['강릉시' '고성군' '동해시' '삼척시' '속초시' '양구군' '양양군' '영월군' '원주시' '인제군' '정선군' '철원군'
 '춘천시' '태백시' '평창군' '홍천군' '화천군' '횡성군' '가평군' '고양시 덕양구' '고양시 일산동구' '고양시 일산서구'
 '과천시' '광명시' '광주시' '구리시' '군포시' '김포시' '남양주시' '동두천시' '부천시' '성남시 분당구'
 '성남시 수정구' '성남시 중원구' '수원시 권선구' '수원시 영통구' '수원시 장안구' '수원시 팔달구' '시흥시'
 '안산시 단원구' '안산시 상록구' '안성시' '안양시 동안구' '안양시 만안구' '양주시' '양평군' '여주시' '연천군'
 '오산시' '용인시 기흥구' '용인시 수지구' '용인시 처인구' '의왕시' '의정부시' '이천시' '파주시' '평택시' '포천시'
 '하남시' '화성시' '거제시' '거창군' '김해시' '남해군' '밀양시' '사천시' '산청군' '양산시' '의령군' '진주시'
 '창녕군' '창원시 마산합포구' '창원시 마산회원구' '창원시 성산구' '창원시 의창구' '창원시 진해구' '통영시' '하동군'
 '함안군' '함양군' '합천군' '경산시' '경주시' '고령군' '구미시' '군위군' '김천시' '문경시' '봉화군' '상주시'
 '성주군' '안동시' '영덕군' '영양군' '영주시' '영천시' '예천군' '울릉군' '울진군' '의성군' '청도군' '청송군'
 '칠곡군' '포항시 남구' '포항시 북구' '광산구' '남구' '동구' '북구' '서구' '달서구' '달성군' '수성구' '중구'
 '대덕구' '유성구' '강서구' '금정구' '기장군' '동래구' '부산진구' '사상구' '사하구' '수영구' '연제구' '영도구'
 '해운대구' '강남구' '강동구' '강북구' '관악구' '광진구' '구로구' '금천구' '노원구' '도봉구' '동대문구' '동작구'
 '마포구' '서대문구' '서초구' '성동구' '성북구' '송파구' '양천구' '

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [10]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [11]:
k = int(len(train_features)*0.9)

x_train = train_features[:k]
y_train = train_target[:k]
x_val = train_features[k:]
y_val = train_target[k:]

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [12]:
import lightgbm as lgbm

In [13]:
train_ds = lgbm.Dataset(x_train, label=y_train)
val_ds = lgbm.Dataset(x_val, label=y_val)

In [14]:
# 파라미터 종류 참고 : https://lightgbm.readthedocs.io/en/latest/Parameters.html
# 파라미터 튜닝 참고 : https://greatjoy.tistory.com/72

params = {
            'learning_rate' : 0.01, # 일반적으로 0.01 ~ 0.1, 성능 높일 때 값 줄여서 진행
            'num_iterations' : 1000, # 디폴트는 100 / 너무 크게하면 과적합 발생 위험
            # 'max_depth' : -1, # 디폴트는 -1 / 0 이하는 제한 없다는 뜻, featureure가 많으면 depth를 크게 가져가야 함
            'boosting': 'gbdt', # 디포트는 gbdt / rf(RandomForest), dart(정확도 중요할 때 사용), goss(샘플링 이용)  / boosting, boosting_type, boost 다 같은 말
            'objective': 'tweedie', # 디폴트는 regression / regression 종류 : regression_l1, huber, fair, poisson, quantile, mape, gamma, tweedie
#             'objective': 'multiclass', # 멀티 클래스로 지정
#             'num_class': 41, # 클래스 수는 업종 수인 41개로 지정
            'tweedie_variance_power': 1.1,
#             'metric': 'softmax',
            'metric': 'custom',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1 # 정규화로 과적합 방지 / 정확도 저하될 수 있기 떄문에 일반적으로는 default 0 그대로 유지
        }

In [15]:
def rmsle_1(y_pred, data):
    y_true = np.array(data.get_label())
    score= np.sqrt(np.square(np.log1p(y_pred + 1) - np.log1p(y_true + 1)).mean())
    return 'rmsle', score, False

In [16]:
# 훈련
model = lgbm.train(params,
                   train_ds,
                   20000,
                   val_ds,
                   verbose_eval = 100,
                   early_stopping_rounds = 100,
                   feval = rmsle_1
                 )

C:\Users\hande\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmsle: 0.0897844
[200]	valid_0's rmsle: 0.0705775
[300]	valid_0's rmsle: 0.062784
[400]	valid_0's rmsle: 0.0581991
[500]	valid_0's rmsle: 0.0552194
[600]	valid_0's rmsle: 0.0530219
[700]	valid_0's rmsle: 0.0514063
[800]	valid_0's rmsle: 0.0501206
[900]	valid_0's rmsle: 0.0491531
[1000]	valid_0's rmsle: 0.0483535
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmsle: 0.0483535


In [1]:
# Model Save
from sklearn.externals import joblib

joblib.dump(model, 'lgbm_gbdt_tweedie_1000epochs.pkl')

C:\Users\hande\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


NameError: name 'model' is not defined

In [23]:
# Model Load
load_model = joblib.load('lgbm_dart_multiclass(41)_1000epochs.pkl')

## 6. 결과 및 결언
## Conclusion & Discussion

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]
seasons       = df_num['season'].unique()
visitors      = df_num['visitor'].unique()
CNT_Categories= df_num['CNT_Category'].unique()
covids        = df_num['covid'].unique()

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                for season in seasons:
                                    for visitor in visitors:
                                        for CNT_Category in CNT_Categories:
                                            for covid in covids:
                                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month, season, visitor, CNT_Category, covid])

temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('HDLY_0715.csv', encoding='utf-8-sig')
submission.head()